In [47]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # to read records completely
import pickle
import numpy as np

## Previous data prep necessary for c) and d)

In [12]:
df = pd.read_pickle('data_clean.pkl')

In [13]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True) # clean some punctuation errors (nissan.)

In [24]:
models = models[~models.brand.isin(['car', 'sedan', 'problem'])]

In [25]:
df.Message.str.contains('said').sum()

370

In [26]:
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

### Replace models with brands

In [27]:
messages = df.Message_words.to_list()

In [28]:
# numpy.where to do the replacement
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [29]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1

In [30]:
print(f'{c} changes of models')

6368 changes of models


## Link attributes to models

In [39]:
# first lets count brand mentions in each record

In [40]:
brands_list = models.brand.drop_duplicates().to_list()

In [41]:
messages = df.Message_words_v2.to_list()

In [43]:
brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

In [44]:
df['Brands_in_message'] = brands_in_message

In [45]:
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))

In [46]:
df['Qty_brands'].value_counts()

# Most messages discuss a single brand
# Also common to compare 2 brands
# 727 messages don't mention any brand (remove them from analysis?)
# Passing a certain treshold (5+), I think it would be better to remove comments since it will be hard to figure out which
# attribute belongs to each brand

0     1291
1     1215
2      755
3      459
4      219
5      109
6       54
7       24
8       10
9        6
11       4
10       3
16       1
13       1
Name: Qty_brands, dtype: int64

In [49]:
df.loc[df.Qty_brands == 0, 'Message_words_v2'].head(10) # to review records with no brand mentions
# most of them are for seeking advice

0                                                                                       [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]
2     [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]
8                                                                                                                                                                                                                                                                                      [id, pass, intrigue, one, reaso

In [50]:
m = df.Message_words_v2[9] #record 9 is a good exaple
brand_dic = {}
for brand in brands_list:
    ix = np.where(m == brand)[0]
    if len(ix) > 0:
        brand_dic[brand] = ix[0]
        

brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

brand_list = list(brand_dic.values()) + [len(m)]

for i, tup in enumerate(brand_dic):
    print(brand_dic[tup])
    print(tup)
    
    if i == 0:
        
        print(m[0:brand_list[1]])
    else:
        print(m[brand_list[i]:brand_list[i+1]])
        
        

2
honda
['would' 'recommend' 'honda' 'honda']
4
toyota
['toyota' 'toyota' 'choices' 'sedan' 'new']
9
nissan
['nissan' 'nissan' 'looks' 'impressive' 'would' 'wait' 'reliability'
 '2002' 'proventhe' 'new']
19
bmw
['bmw' '525' 'another' 'nice' 'car' 'rated' 'highly' 'many' 'auto'
 'publications']


In [51]:
# now we run it for all records

brand_reviews = []

for m in df.Message_words_v2:

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]

    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]


    review={}
    for i, tup in enumerate(brand_dic):
        #print(brand_dic[tup])
        #print(tup)

        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    brand_reviews.append(review)

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

In [53]:
len(brand_reviews) # each record becomes a single dictionary with all its brand mentions and linked words

4151

In [57]:
brand_reviews[9]

{'honda': array(['would', 'recommend', 'honda', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'nissan', 'looks', 'impressive', 'would', 'wait',
        'reliability', '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [58]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
review_accum = {i:[] for i in brands_list}

In [59]:
for brand in brands_list:
    for review in brand_reviews:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum[brand].extend(review[single_review])

In [ ]:
## from here, we are ready to start counting attributes per brand
## we also need to consider replacing some similar attributes before doing the final count 
## I will do one example with Audi but it could become a function and run in a loop to all the brands 

In [ ]:
# first lets discover popular attributes regardless of brands

all_attributes = []

for i in list(review_accum.values()):
    all_attributes.extend(i)
    
words = pd.Series(all_attributes)

words.value_counts().head(40) # this shows the frequencies of each word
# the most "rustic" approach would be to literally print them in order and take note of the ones that can be considered "attributes"

In [84]:
# once we have the list of the 5 most popular attributes, my guess is we have to calculate the uplift scores of said attribute
# against the 5 top brands found in letter A). Then the uplift should tell us which brand is more related to which attribute

In [110]:
# let's say "reliability" is one of the popular attributes 
# and the top 5 brands are: honda, nissan, audi, bwm, toyota
# let's get the inputs necessary to calculate uplift of reliability in nissan

In [112]:
n=0
for i in  df.Brands_in_message:
    if 'nissan' in i:
        n += 1
n
#nissan appears in 525 rows 

525

In [113]:
df_nissan = pd.DataFrame(review_accum['nissan'], columns = ['word'])
(df_nissan.word == 'reliability').sum() # 73 mentions of "relibaility" in honda

# and 28 times the term "reliability" was mentioned alongside a nissan

28

In [114]:
# how about honda?

n=0
for i in  df.Brands_in_message:
    if 'honda' in i:
        n += 1
print(n)


df_honda = pd.DataFrame(review_accum['honda'], columns = ['word'])
(df_honda.word == 'reliability').sum() # 73 mentions of "relibaility" in honda


924


73

In [115]:
73/924 # not the exact uplift formula but a quick validation

0.07900432900432901

In [116]:
28/525 # vs nissan

0.05333333333333334

In [117]:
# honda has more prevalence as a reliable brand than nissan (pending to adjust for uplift real formula)

to consider:

right now the code counts all the appearances of an attribute, even if they come from the same record; I think that is not the exact definition of the uplift formula. I don't think it will change the numbers very much, but would be good to refine it if we have the time